In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [54]:
num_users = 2
start_date = "2020-01-01"
end_date = "2024-06-28"
output_file="stock_trade.csv"

# 입력된 날짜 문자열을 datetime 객체로 변환
start_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")

In [58]:
users_id = []
for i in range(num_users):
    users_id.append(f'user_{i+1}')
users_id

['user_1', 'user_2']

In [4]:
# 지표 데이터 가져오기
stock_data = pd.read_excel('./data/index_Infomation_v0.1.xlsx')
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data = stock_data[['Date', 'samsung_Close', 'apple_Close', 'dollar_to_won_Close']]
stock_data

,Date,samsung_Close,apple_Close,dollar_to_won_Close
0,2014-11-01,24880.0,27.000000,1053.80004
1,2014-11-02,24880.0,27.000000,1053.80004
2,2014-11-03,24700.0,27.350000,1051.19995
3,2014-11-04,24340.0,27.150000,1081.59997
4,2014-11-05,24040.0,27.215000,1076.90002
...,...,...,...,...
3525,2024-06-26,81300.0,213.250000,1390.89001
3526,2024-06-27,81600.0,214.100006,1393.38000
3527,2024-06-28,81500.0,210.619995,1386.94995
3528,2024-06-29,81500.0,210.619995,1386.94995


In [5]:
# 주식 가격 함수
def buy_stock(buy_date, buy_stock, buy_qty=1):
    if buy_stock == 'apple':
        stock = stock_data.loc[stock_data['Date'] == buy_date, 'apple_Close'].values[0]
        exchange_rate = stock_data.loc[stock_data['Date'] == buy_date, 'dollar_to_won_Close'].values[0]
        buy_cost =  stock * exchange_rate
        return buy_cost * buy_qty
    
    elif buy_stock == 'samsung':
        stock = stock_data.loc[stock_data['Date'] == buy_date, 'samsung_Close'].values[0]
        exchange_rate = stock_data.loc[stock_data['Date'] == buy_date, 'dollar_to_won_Close'].values[0]
        buy_cost =  stock
        return buy_cost * buy_qty

In [6]:
buy_stock("2024-06-27", "samsung")

81600.0

In [7]:
# 주식 거래
trade_type = ['buy', 'sell']
stock_name = ['samsung', 'apple']

In [12]:
# # 주식 구매/판매 내역 생성 (중복일 허용)
# for user_id in users_id:
    
#     # 초기 날짜 설정
#     current_date = start_date
    
#     while current_date <= end_date:
        
#         if current_date.day == 1:
#             num_trades_month = np.random.randint(0, 10)  # 한달에 0~10회 거래 설정

#             for qty in range(num_trades_month):
#                 lastday_of_month = (current_date.replace(month=current_date.month % 12 + 1, day=1) - timedelta(days=1)).day  # 월말일
#                 transaction_day = random.randint(1, lastday_of_month)  # 랜덤 날짜 선택
#                 transaction_date = current_date.replace(day=transaction_day)  # 랜덤day를 기반으로 datetime형식 저장
#                 stock_choice = random.choice(list(stock_name))  # 거래 주식 랜덤 선택
#                 trade_choice = random.choice(['buy', 'sell'])  # 거래 타입 랜덤 선택

#                 if trade_choice == 'buy':
#                     num_buy = random.randint(1, 50) # 1~50주 거래 수량 랜덤 설정
#                     cost = buy_stock(transaction_date, stock_choice)
#                     amount = buy_stock(transaction_date, stock_choice, num_buy)

#                     stock_trade_data['User_ID'].append(user_id)
#                     stock_trade_data['Date'].append(transaction_date)
#                     stock_trade_data['Stock_name'].append(stock_choice)
#                     stock_trade_data['Buy_qty'].append(num_buy)
#                     stock_trade_data['Stock_cost'].append(cost)
#                     stock_trade_data['Amount'].append(amount)
#                     stock_trade_data['Fixed'] = False
                    
#                     # 구매한 주식을 사용자 보유 주식에 추가
#                     user_stocks[user_id][stock_choice] += num_buy


#                 elif trade_choice == 'sell':
#                     max_sell_qty = user_stocks[user_id][stock_choice]  # 보유 주식 수
                    
#                     if max_sell_qty > 0:
#                         num_sell = random.randint(1, max_sell_qty)  # 1~보유 주식 수 만큼 랜덤 설정
#                         cost = buy_stock(transaction_date, stock_choice)
#                         amount = buy_stock(transaction_date, stock_choice, num_sell)

#                         stock_trade_data['User_ID'].append(user_id)
#                         stock_trade_data['Date'].append(transaction_date)
#                         stock_trade_data['Stock_name'].append(stock_choice)
#                         stock_trade_data['Buy_qty'].append(-num_sell)
#                         stock_trade_data['Stock_cost'].append(-cost)
#                         stock_trade_data['Amount'].append(-amount)
#                         stock_trade_data['Fixed'] = False

#                         # 판매한 주식을 사용자 보유 주식에서 차감
#                         user_stocks[user_id][stock_choice] -= num_sell

#         current_date += timedelta(days=1)

In [62]:
# 주식 구매/판매 내역 생성

# 데이터프레임 초기화
stock_trade_data = {
    'User_ID': [],
    'Date': [],
    'Stock_name': [],
    'Buy_qty' : [],
    'Stock_cost' : [],
    'Amount': [],
    'Fixed' : False
}

# 사용자별 보유 주식 딕셔너리 초기화
user_stocks = {user_id: {stock: 0 for stock in stock_name} for user_id in users_id}


for user_id in users_id:

    # 초기 날짜 세팅
    current_date = start_date
        
    while current_date <= end_date:
        
        if current_date.day == 1:
            num_trades_month = np.random.randint(0, 10)  # 한달에 0~10회 거래 설정
            lastday_of_month = (current_date.replace(month=current_date.month % 12 + 1, day=1) - timedelta(days=1)).day  # 월말일
            transaction_days = random.sample(range(1, lastday_of_month + 1), num_trades_month)
            transaction_days.sort()

            for transaction_day in transaction_days:
                transaction_date = current_date.replace(day=transaction_day)  # 랜덤day를 기반으로 datetime형식 저장
                stock_choice = random.choice(list(stock_name))  # 거래 주식 랜덤 선택
                trade_choice = random.choice(['buy', 'sell'])  # 거래 타입 랜덤 선택

                if trade_choice == 'buy':
                    num_buy = random.randint(1, 50)  # 1~50주 거래 수량 랜덤 설정
                    cost = buy_stock(transaction_date, stock_choice)
                    amount = buy_stock(transaction_date, stock_choice, num_buy)

                    stock_trade_data['User_ID'].append(user_id)
                    stock_trade_data['Date'].append(transaction_date)
                    stock_trade_data['Stock_name'].append(stock_choice)
                    stock_trade_data['Buy_qty'].append(num_buy)
                    stock_trade_data['Stock_cost'].append(cost)
                    stock_trade_data['Amount'].append(amount)
                    stock_trade_data['Fixed'] = False
                    
                    # 구매한 주식을 사용자 보유 주식에 추가
                    user_stocks[user_id][stock_choice] += num_buy
                    

                elif trade_choice == 'sell':
                    max_sell_qty = user_stocks[user_id][stock_choice]  # 보유 주식 수

                    if max_sell_qty > 0:  # 보유 주식이 있는 경우에만 판매
                        num_sell = random.randint(1, max_sell_qty)  # 1~보유 주식 수 만큼 랜덤 설정
                        cost = buy_stock(transaction_date, stock_choice)
                        amount = buy_stock(transaction_date, stock_choice, num_sell)

                        stock_trade_data['User_ID'].append(user_id)
                        stock_trade_data['Date'].append(transaction_date)
                        stock_trade_data['Stock_name'].append(stock_choice)
                        stock_trade_data['Buy_qty'].append(-num_sell)
                        stock_trade_data['Stock_cost'].append(-cost)
                        stock_trade_data['Amount'].append(-amount)
                        stock_trade_data['Fixed'] = False

                        # 판매한 주식을 사용자 보유 주식에서 차감
                        user_stocks[user_id][stock_choice] -= num_sell
                    

        current_date += timedelta(days=1)

In [63]:
# 데이터프레임 생성
stock_trade_df = pd.DataFrame(stock_trade_data)
stock_trade_df

,User_ID,Date,Stock_name,Buy_qty,Stock_cost,Amount,Fixed
0,user_1,2020-01-01,apple,7,84699.669567,5.928977e+05,False
1,user_1,2020-01-06,samsung,14,55500.000000,7.770000e+05,False
2,user_1,2020-01-11,samsung,-12,-59500.000000,-7.140000e+05,False
3,user_1,2020-01-20,samsung,42,62400.000000,2.620800e+06,False
4,user_1,2020-01-23,samsung,4,60800.000000,2.432000e+05,False
...,...,...,...,...,...,...,...
495,user_2,2024-06-14,samsung,5,79600.000000,3.980000e+05,False
496,user_2,2024-06-15,samsung,43,79600.000000,3.422800e+06,False
497,user_2,2024-06-18,apple,-12,-295518.768461,-3.546225e+06,False
498,user_2,2024-06-21,samsung,40,80000.000000,3.200000e+06,False


In [64]:
# 데이터프레임 CSV 파일로 저장
stock_trade_df.to_csv(f'.\data\{output_file}', index=False, encoding='cp949')